# Experiment 2: Syntactic Integrity (The "Passive Voice" Collapse)

This notebook visualizes the finding that Synthetic Models (like Phi-3) exhibit a spectral collapse when processing complex syntax (Passive Voice).

**Data Source**: Pre-computed results from `results/phi3_syntactic.json`.

**To generate fresh results**, run:
```bash
python scripts/reproduce.py --experiment scar --model Phi-3
```

In [ ]:
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Setup Plotting
plt.rcParams['font.family'] = 'serif'
sns.set_theme(style="white", font="serif")

## Load Pre-Computed Results

In [ ]:
with open('../results/phi3_syntactic.json', 'r') as f:
    data = json.load(f)

# Extract Active vs Passive for English
active_data = [x for x in data if x.get('lang') == 'en' and x.get('type') == 'active']
passive_data = [x for x in data if x.get('lang') == 'en' and x.get('type') == 'passive']

print(f"Active samples: {len(active_data)}, Passive samples: {len(passive_data)}")

## Extract Fiedler Trajectories

In [ ]:
def get_mean_trajectory(items, metric='fiedler_value'):
    trajs = []
    for item in items:
        traj = [step.get(metric) for step in item.get('trajectory', [])]
        trajs.append(traj)
    return np.nanmean(trajs, axis=0) if trajs else []

fvals_active = get_mean_trajectory(active_data)
fvals_passive = get_mean_trajectory(passive_data)

layers = np.arange(len(fvals_active))

## Visualize the "Scar"

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(layers, fvals_active, label='Active Voice', linewidth=2.5, color='blue')
plt.plot(layers, fvals_passive, label='Passive Voice', linewidth=2.5, color='red', linestyle='--')

# Find the collapse layer
delta = fvals_passive - fvals_active
collapse_layer = np.argmin(delta)
collapse_val = delta[collapse_layer]

plt.annotate(f'Synthetic Scar\n(Delta: {collapse_val:.2f})', 
             xy=(collapse_layer, fvals_passive[collapse_layer]), 
             xytext=(collapse_layer + 5, fvals_passive[collapse_layer] + 0.1),
             arrowprops=dict(facecolor='black', shrink=0.05),
             fontsize=12, fontweight='bold')

plt.title('Syntactic Integrity: Phi-3', fontweight='bold')
plt.xlabel('Layer')
plt.ylabel('Fiedler Value (Connectivity)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()